In [1]:
# Install latest pre-release version of bigdl-dllib with spark3
# Find the latest bigdl-dllib with spark3 from https://sourceforge.net/projects/analytics-zoo/files/dllib-py-spark3/ and intall it
!pip install https://sourceforge.net/projects/analytics-zoo/files/dllib-py-spark3/bigdl_dllib_spark3-0.14.0b20211107-py3-none-manylinux1_x86_64.whl

exit() # restart the runtime to refresh installed pkg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━ 67.4/93.9 MB 1.8 MB/s eta 0:00:15
ERROR: Exception:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_vendor/urllib3/response.py", line 438, in _error_catcher
    yield
  File "/usr/local/lib/python3.11/dist-packages/pip/_vendor/urllib3/response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_vendor/urllib3/response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_vendor/cachecontrol/filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
                  ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/http/client.py", line 473, in read
    s = self.fp.read(amt)
        ^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/socket.py", line 718, in readin

In [2]:
pip install --upgrade bigdl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 20.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 69.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (pyproject.toml) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import lit
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from bigdl.dllib.nncontext import *
from bigdl.dllib.keras.layers import *
from bigdl.dllib.keras.models import *
#import bigdl.dllib.keras.Sequential
from bigdl.dllib.nnframes import *
from bigdl.dllib.nn.criterion import *

In [ ]:
sc = init_nncontext(cluster_mode="local") # run in local mode
spark = SparkSession(sc)

Current pyspark location is : /usr/local/lib/python3.11/dist-packages/pyspark/__init__.py
Start to getOrCreate SparkContext
pyspark_submit_args is:  --driver-class-path /usr/local/lib/python3.11/dist-packages/bigdl/share/dllib/lib/bigdl-dllib-spark_3.1.2-0.14.0-SNAPSHOT-jar-with-dependencies.jar pyspark-shell 
Successfully got a SparkContext


IMPORTAR ARCHIVO KAGGEL

In [ ]:
import os
os.environ["KAGGLE_CONFIG_DIR"] = "/content/drive/MyDrive/kaggle"

from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()


In [ ]:

# Descargar el dataset
dataset_name = 'paultimothymooney/chest-xray-pneumonia'
output_dir = 'chest_xray_pneumonia'

# Descargar y descomprimir
api.dataset_download_files(dataset_name, path=output_dir, unzip=True)

print(f"Dataset descargado y descomprimido en la carpeta: {output_dir}")


Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
Dataset descargado y descomprimido en la carpeta: chest_xray_pneumonia


In [ ]:
df_train = spark.read.format('csv').options(inferSchema=True, header=True).load('caiis-dogfood-day-2020/train.csv')

CÓDIGO CHATGPT


In [ ]:
# Importar librerías necesarias
from pyspark.sql import SparkSession
from bigdl.dllib.nncontext import init_nncontext
from bigdl.dllib.keras.models import Sequential
from bigdl.dllib.keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout
from bigdl.dllib.optim.optimizer import Adam
from pyspark.ml.image import ImageSchema
import os
import numpy as np

In [ ]:
spark = SparkSession.builder \
    .appName("PneumoniaDetection") \
    .getOrCreate()

sc = init_nncontext("PneumoniaDetection")


In [ ]:
# Función para preprocesar datos
def preprocess_data(base_dir):
    """
    Carga y preprocesa imágenes desde los directorios de entrenamiento y prueba.
    """
    train_dir = os.path.join(base_dir, "train")
    test_dir = os.path.join(base_dir, "test")

    # Leer imágenes como DataFrames de Spark
    train_df = ImageSchema.readImages(train_dir)
    test_df = ImageSchema.readImages(test_dir)

    # Extraer rutas y etiquetas
    train_paths = train_df.select("image.origin").rdd.map(lambda row: row[0]).collect()
    train_labels = [1 if "PNEUMONIA" in path else 0 for path in train_paths]

    test_paths = test_df.select("image.origin").rdd.map(lambda row: row[0]).collect()
    test_labels = [1 if "PNEUMONIA" in path else 0 for path in test_paths]

    return train_paths, train_labels, test_paths, test_labels


In [ ]:
# Construir el modelo con BigDL
def build_model():
    """
    Define un modelo CNN usando BigDL Keras API.
    """
    model = Sequential()
    model.add(Convolution2D(32, 3, 3, activation="relu", input_shape=(3, 64, 64)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation="softmax"))
    return model

In [ ]:
# Convertir imágenes a tensores y normalizar
def process_images(paths):
    """
    Convierte imágenes en tensores normalizados.
    """
    from PIL import Image
    images = []
    for path in paths:
        img = Image.open(path).resize((64, 64))
        img_array = np.array(img, dtype=np.float32) / 255.0
        images.append(img_array.transpose((2, 0, 1)))  # Reordenar a (canales, alto, ancho)
    return np.array(images)


In [ ]:
# Directorio base del dataset
base_dir = "chest_xray_pneumonia"

# Preprocesar datos
train_paths, train_labels, test_paths, test_labels = preprocess_data(base_dir)

# Procesar imágenes y etiquetas
train_images = process_images(train_paths)
test_images = process_images(test_paths)

# Convertir etiquetas a formato compatible
train_labels = np.array(train_labels, dtype=np.int32)
test_labels = np.array(test_labels, dtype=np.int32)

# Construir modelo
model = build_model()

# Configurar optimizador
model.compile(optimizer=Adam(), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Entrenar modelo
model.fit(train_images, train_labels, batch_size=32, nb_epoch=10, validation_data=(test_images, test_labels))

# Evaluar modelo
evaluation = model.evaluate(test_images, test_labels)
print(f"Evaluación del modelo: {evaluation}")

AttributeError: '_ImageSchema' object has no attribute 'readImages'